# PMR3508 - Aprendizado de Máquina e Reconhecimento de Padrões
**Tarefa 3 - Regressão**

HASH: f20977e697


## Table of Contents

- [Introduction](#introduction)
- [Initial Data Exploration](#initial)
- [Future Engeneering](#future)
- [Deeper Data Exploration](#deeper)
- [Adjustments before starting the regression tests](#adjustments)
- [Linear Regression](#classify0)
- [Ridge Regression](#classify1)
- [Lasso Regression](#classify2)
- [Elastic Net Regression](#classify3)
- [Random Forest Regressor](#classify4)
- [KNN Regression](#classify5)
- [Conclusions](#conclusions)
- [References](#references) 

## 1. Introduction

In this kernel, we will build a model that predicts the median price of a house in a California region.
The goal of this task is that we test all regression models known to us and choose the most suitable for this task.

### Environment Setting

Before we start we need set up the environment, please make sure those packages are installed in your computer when you copy this code to the local file.

In [ ]:
#Importing stuff that I will need in this notebook

from IPython.display import Image
from IPython.core.display import HTML
import seaborn as sns 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import matplotlib
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from geopy.distance import geodesic, vincenty
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.model_selection import cross_val_score, train_test_split, KFold
from itertools import product
from sklearn.metrics import make_scorer,  r2_score
from scipy import stats
from scipy.stats import norm, skew
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV, BayesianRidge, Lasso
from scipy import stats
from sklearn.pipeline import make_pipeline
from geopy.geocoders import Nominatim
from sklearn import feature_extraction, model_selection, metrics
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### Read Data File

The first task is to read data from .csv file. Most of the features are frequency of some especif word.

In [ ]:
train = pd.read_csv( "../input/califdata/train.csv")
testTOsubmit = pd.read_csv("../input/califdata/test.csv" )

In [ ]:
print ("train size: ", train.shape)
print ("test size: ", testTOsubmit.shape)

In [ ]:
train.head()

 - As we can see above, the features are related to an especif house and our target is the house price. Also, below, we can see that the database does not have missing data (which is good for a better analysis).

In [ ]:
train.info()

In [ ]:
train.isnull().sum()

In [ ]:
testTOsubmit.info()

In [ ]:
testTOsubmit.isnull().sum()

## 2. Initial data exploration

 In this section, we will explore the data in order to understand what each of them means in the database and how they relate to the target variable.

In [ ]:
train.describe()

 - Firstly, we will explore the data and your relation with the target

In [ ]:
def grafico(col):
    plt.figure(figsize=(16,5))
    plt.subplot(121)
    sns.distplot(col);

    plt.subplot(122)
    sns.boxplot(col);

In [ ]:
# Plot for target variable

grafico(train['median_house_value'])

In [ ]:
# Let's check out the numerical variables at one place in a grid like plot

df = train.dropna()

In [ ]:
# Select only numerical "independent" columns

numerical_columns = df.drop(['Id', 'median_house_value', 'longitude', 'latitude'], axis=1).columns

plt.figure(figsize=(27,12))
for k,v in enumerate(numerical_columns):
    plt.subplot(2,4,k+1)
    sns.distplot(df[v])
    plt.tight_layout();

 - Let's plot them individually and check the presence of ouliers with the help of boxplots

In [ ]:
grafico(train['median_age']);

In [ ]:
grafico(train['total_rooms']);

In [ ]:
grafico(train['total_bedrooms']);

In [ ]:
grafico(train['population']);

In [ ]:
grafico(train['households']);

In [ ]:
grafico(train['median_income']);

 - Let's get the correlation between two columns

In [ ]:
corr_matrix = train.corr()
print(corr_matrix)

In [ ]:
# And, here, more information about the target related to the others columns

corr_matrix["median_house_value"].sort_values(ascending=False)

 - As we can se above, 'median_income' is very related to the 'median_house_value'. It is easy to understand this high relationship between the two variables, because the more money the person has, the better the home and standard of living he will want to have. Therefore, more expensive will be the value and his home.

In [ ]:
# Ploting the correlation between two columns

attributes = ["median_house_value", "median_income", "total_rooms",
              "median_age"]
scatter_matrix(train[attributes], figsize=(18, 12))

 - Just analyzing better the relationship between median income and median house value (let's plot this graph bigger)

In [ ]:
train.plot(kind='scatter', x='median_income', y='median_house_value', alpha=0.2, figsize=(10,10))

 - As we can see above, there are some "horizontal lines" in the plot, one clear one at  500,000, other at 450,000 and another one at 350,000, and a few other ones. If we cannot figure out the reason, removing those data points might be a good idea before feeding the data to the algorithms.

 - Here, let's get a deeply analysis between the relationship of coordinates and the median house value

In [ ]:
plt.figure(figsize=(15,5))
plt.subplots_adjust(hspace = .25)
plt.subplot(1,2,1)
plt.title('Corelation between longtitude and median_house_value')
plt.xlabel('longitude',fontsize=12)
plt.ylabel('median_house_value',fontsize=12)
plt.scatter(train['longitude'].head(100),train['median_house_value'].head(100),color='g')
plt.subplot(1,2,2)
plt.title('Corelation between latitude and median_house_value')
plt.xlabel('latitude',fontsize=12)
plt.ylabel('median_house_value',fontsize=12)
plt.scatter(train['latitude'].head(100),train['median_house_value'].head(100),color='r')

 - As we can see above, in some specific latitudes and longitudes, the median house value can be very diverse (these are, probably, big cities or populous places).

 - Now, let's see the median house value distribution.

In [ ]:
sns.distplot(train['median_house_value'] , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(train['median_house_value'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('Median House Value Distribution')

fig = plt.figure()
res = stats.probplot(train['median_house_value'], plot=plt)
plt.show()

 - As we can see, the higher frequency of house prices is between 10000 and 20000 and this is a Skewed Dataset (with a positive skew).

## 3. Future Engeneering

 Here, we will use the domain knowledge of the data to create features that make machine learning algorithms work. Our intent is to increase the predictive power of machine learning algorithms by creating features from raw data that help facilitate the machine learning process.

 - Let's join the 'train' and 'test' for the future adjustments be made in both (then we can separate).

In [ ]:
testTOsubmit['median_house_value'] = np.nan

In [ ]:
data = train.append(testTOsubmit, ignore_index = True)

In [ ]:
data.info()

 - In this section, we will create new features with the relation of some of them

In [ ]:
data["rooms_per_household"] = data["total_rooms"]/data["households"]

In [ ]:
data["bedrooms_per_room"] = data["total_bedrooms"]/data["total_rooms"]

In [ ]:
data["population_per_household"]= data["population"]/data["households"]

In [ ]:
data["population_per_room"]= data["population"]/data["total_rooms"]

 ### Geoespatial Feature Engeneering 
 
 - In this subsection, we use other datas that are useful to complement and understand the locality of the californian houses and their relationship with the prices

In [ ]:
# Reading these data files

# A list of the latitude and longitude of cities in California
city_lat_long = pd.read_csv( '../input/califdata/cal_cities_lat_long.csv')

# Historical population data for cities in California (including the year 1990, which is the year the original housing price data)
city_pop_data = pd.read_csv( '../input/califdata/cal_populations_city.csv')

# Historical population data for countries in California (including the year 1990, which is the year the original housing price data)
county_pop_data = pd.read_csv( '../input/califdata/cal_populations_county.csv')

 - Now, let's iterates through the coordinates dataframe and builds a dictonary with the city names as the keys and a tuple of the corresponding (latitude, longitude) as the values.

In [ ]:
city_coords = {}
for dado in city_lat_long.iterrows():
    row = dado[1]
    if row['Name'] not in city_pop_data['City'].values:   
        continue           
    else: 
        city_coords[row['Name']] = (float(row['Latitude']), float(row['Longitude']))

 - Here, we will determine the distance between two sets of coordinates and, then, try all pairwise comparisons and figure out which city is the closest.

In [ ]:
def closest_point(location, location_dict):
    closest_location = None
    for city in location_dict.keys():
        distance = vincenty(location, location_dict[city]).kilometers
        if closest_location is None:
            closest_location = (city, distance)
        elif distance < closest_location[1]:
            closest_location = (city, distance)
    return closest_location

In [ ]:
# Let's use the April 1990 population data below, because the California Housing was collected at this time
city_pop_dict = {}
for dado in city_pop_data.iterrows():
    row = dado[1]
    city_pop_dict[row['City']] =  row['pop_april_1990']

    
# For big cities, we will separate in an another coordinate dictonary
big_cities = {}
for key, value in city_coords.items():
    if city_pop_dict[key] > 500000:
        big_cities[key] = value

In [ ]:
# Adding the data relating to the points to the closest city

data['close_city'] = data.apply(lambda x: closest_point((x['latitude'],x['longitude']),city_coords), axis = 1)
data['close_city_name'] = [x[0] for x in data['close_city'].values]
data['close_city_dist'] = [x[1] for x in data['close_city'].values]
data['close_city_pop'] = [city_pop_dict[x] for x in data['close_city_name'].values]

In [ ]:
data = data.drop('close_city', axis=1)

In [ ]:
# Adding the data relating to the points to the closest big city

data['big_city'] = data.apply(lambda x: closest_point((x['latitude'],x['longitude']),big_cities), axis = 1)
data['big_city_name'] = [x[0] for x in data['big_city'].values]
data['big_city_dist'] = [x[1] for x in data['big_city'].values]

In [ ]:
data = data.drop('big_city', axis=1)

- We find in the kaggle the same database with an additional feature (called ocean_proximity), so, let's use it! For integrate this feature in our database, we will merge by the location of each house

## 4. Deeper data exploration

 In this moment, we will continue exploring the data with the feature engeneering adjusted.
 
( OBS.: Não sei o que ocorreu, porém, não consigo colocar imagens no Kaggle, esse erro só ocorre dentro da pataforma...)

### Data Visualization

 - Ploting in the map of California our location and population of housing districts, along with a heatmap to show where homes are the most expensive

In [ ]:
# Importing geographical image of the State of California
california_img = Image(url= "https://ibb.co/eOSUtL")
housing_plot = data[['longitude','population','latitude',
                      'close_city_name','big_city_name','big_city_dist','median_house_value']]

#Ploting the map
housing_plot.plot(kind='scatter', x='longitude', y='latitude', alpha=0.4,
                  s=housing_plot['population']/100, label='population', figsize=(10,7),
                  c='median_house_value', cmap=plt.get_cmap('jet'), colorbar=True)


plt.imshow(california_img, extent=[-124.55, -113.80, 32.45, 42.05], alpha=0.5)
plt.ylabel("Latitude", fontsize=14)
plt.xlabel("Longitude", fontsize=14)
plt.legend()
plt.show()

 - Now, we will plot a map that provides a visualization of three of the features added to the data. The black lines represent the vectors connecting the districts to the nearest town. Based on these lines, the distance to the nearest town (length of black line), the population of the nearest town (point the black line arrives at) and a categorical for the name of the nearest town were assigned to the district.These features help to capture the general trend of houses getting cheaper the further away from downtown one gets (a fairly universal pattern... with plenty of exceptions).

In [ ]:
# Graph of vectors connecting points to their nearest city

city_lat_long.plot(kind='scatter', x='Longitude', y='Latitude',  alpha=0.4,
                   s=housing_plot['population']/100, label='population', figsize=(10,7))

for line in data.iterrows():
    dat = line[1]
    x1 = dat['longitude']
    y1 = dat['latitude']
    p2 = city_coords[dat['close_city_name']]
    x2 = p2[1]
    y2 = p2[0]
    plt.plot([x1,x2],[y1, y2], 'k-',linewidth=0.1)

plt.imshow(california_img, extent=[-124.55, -113.80, 32.45, 42.05], alpha=0.5)
plt.ylabel("Latitude", fontsize=14)
plt.xlabel("Longitude", fontsize=14)
plt.show()

 - Now, we will analyse the houses and their distances to the big cities (San Francisco, San Jose, San Diego or Los Angeles). We used a cutoff for 'big city' of 500,000 people (that is quite big to 1990). I decided to add these features in because the proximity to a large city is somthing I intuitively thought would impact a house price (this definitely separates which are cosmopolitan houses or not).

In [ ]:
# Ploting map of the vectors connecting districts to the nearest major city

city_lat_long.plot(kind='scatter', x='Longitude', y='Latitude',  alpha=0.4,
                   s=housing_plot['population']/100, label='population', figsize=(10,7))

for line in data.iterrows():
    dat = line[1]
    x1 = dat['longitude']
    y1 = dat['latitude']
    p2 = big_cities[dat['big_city_name']]
    x2 = p2[1]
    y2 = p2[0]
    plt.plot([x1,x2],[y1, y2], 'k-',linewidth=0.1)

plt.imshow(california_img, extent=[-124.55, -113.80, 32.45, 42.05], alpha=0.5)
plt.ylabel("Latitude", fontsize=14)
plt.xlabel("Longitude", fontsize=14)
plt.show()

 - As we can see above, this clustering also has an interesting side effect of effectively splitting California into quarters. So having the categorical San Francisco as the closest big city is roughly equivalent to being from the northern 1/4 of the state, while the San Diego category is roughly equivalent to the bottom 1/4 of California. So this feature may be adding some extra structure to the data in ways I had not initially intended.

 - Here, let's plot a correlation matrix to see the correlation coefficients between variables. Each cell in the table shows the correlation between two variables.

In [ ]:
# Plot a correlation matrix
corrmat = data.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);

## 5. Adjustments before starting the regression tests

### Enconde text attributes

 - Most ML algorithms work with numbers better. Therefore, we will convert text attributes into numerical attributes.

In [ ]:
encoder = LabelEncoder()

In [ ]:
# Encoder the ocean close city name
data['close_city_name'] = encoder.fit_transform(data['close_city_name'])

# Encoder the ocean big city name
data['big_city_name'] = encoder.fit_transform(data['big_city_name'])

### log Transformation
 - The best way to fix the Skewed Dataset is to perform a log transform of the same data, with the intent to reduce the skewness. After taking logarithm of the same data the curve seems to be normally distributed.

In [ ]:
#log transform skewed numeric features

skewed = data.apply(lambda x: skew(x.dropna())) #compute skewness
skewed = skewed[skewed > 0.75]
skewed = skewed.index

data[skewed] = np.log1p(data[skewed])

In [ ]:
data.head(10)

### Feature Scaling

 - Typically, ML algorithms don't perform well when the input numerial attributes have very different scales. So, let's see if the dataset has very different scales between its features.

In [ ]:
data.describe()

 - As we can see above, in this housing dataset, you can see, for example, 'median_income' ranges from 4999
 to 150001 while 'total_rooms' is between 2 and 39320 (note that scaling the target values is typically not required). Let's do the feature scaling now!

In [ ]:
scaler = MinMaxScaler()

In [ ]:
# Here, we will separate the 'Id' and 'median_house_value' to preservate its real number and apply the feature scaling

dataID = data['Id']
dataMEDIAN = data['median_house_value']
data_scaled = data.apply(lambda x:(x.astype(float) - min(x))/(max(x)-min(x)), axis = 0)
data_scaled.head(10)
data = data_scaled
data['Id2'] = dataID 
data['median_house_value2'] = dataMEDIAN 

In [ ]:
data.head()

 - As we can see above, everything has occurred correctly and we are with our features between 0 and 1

### Split Data

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
#Separate what is test to submit of the rest

SPLIT = data[data['median_house_value'].notnull()]
testTOsubmit = data[data['median_house_value'].isnull()]

In [ ]:
#Split the train and test to measure the quality of each type of regression

train, test = train_test_split(SPLIT, test_size=0.2)

In [ ]:
trainID = train['Id2']
trainMEDIAN = train['median_house_value2']
testID = test['Id2']
testMEDIAN = test['median_house_value2']
testTOsubmitID = testTOsubmit['Id2']
testTOsubmitMEDIAN = testTOsubmit['median_house_value2']

In [ ]:
train = train.drop('Id2', axis=1)
train = train.drop('median_house_value2', axis=1)
test = test.drop('Id2', axis=1)
test = test.drop('median_house_value2', axis=1)
testTOsubmit = testTOsubmit.drop('Id2', axis=1)
testTOsubmit = testTOsubmit.drop('median_house_value2', axis=1)

 - Just confirming that everything is right!

In [ ]:
train

In [ ]:
test

In [ ]:
testTOsubmit

In [ ]:
print ("train size: ", train.shape)
print ("test size: ", test.shape)
print ("test to submit size: ", testTOsubmit.shape)

 - Here, we will attribute some variables

In [ ]:
Ytrain = train['median_house_value']
Xtrain = train.drop('median_house_value', axis=1)
Ytest = test['median_house_value']
Xtest = test.drop('median_house_value', axis=1)

In [ ]:
Xtrain.shape,Xtest.shape,Ytrain.shape,Ytest.shape

## 6. Linear Regression Model (without Regularization)
 - In statistics, linear regression is a linear approach to modelling the relationship between a scalar response (or dependent variable) and one or more explanatory variables (or independent variables).

In [ ]:
# Defining the function of cross vaidation for test and train

n_folds = 5
scorer = make_scorer(mean_squared_error,greater_is_better = False)
def rmse_CV_train(model):
    kf = KFold(n_folds,shuffle=True,random_state=42).get_n_splits(train.values)
    rmse = np.sqrt(-cross_val_score(model,Xtrain,Ytrain,scoring ="neg_mean_squared_error",cv=kf))
    return (rmse)
def rmse_CV_test(model):
    kf = KFold(n_folds,shuffle=True,random_state=42).get_n_splits(train.values)
    rmse = np.sqrt(-cross_val_score(model,Xtest,Ytest,scoring ="neg_mean_squared_error",cv=kf))
    return (rmse)

In [ ]:
# Getting the predictions to Linear Regression

lr = LinearRegression()
lr.fit(Xtrain, Ytrain)
test_pre = lr.predict(Xtest)
train_pre = lr.predict(Xtrain)

In [ ]:
# Calculating the Linear Regression RMSE

print("Linear Regression RMSE on Training set :", rmse_CV_train(lr).mean())
print("Linear Regression RMSE on Test set :", rmse_CV_test(lr).mean())

In [ ]:
score = cross_val_score(lr,Xtrain, Ytrain, cv = 5)
print('Score is: '+ str(np.mean(score)))

In [ ]:
# Plot between predicted values and residuals

plt.scatter(train_pre, train_pre - Ytrain, c = "red",  label = "Training data")
plt.scatter(test_pre,test_pre - Ytest, c = "green",  label = "Validation data")
plt.title("Linear regression")
plt.xlabel("Predicted values")
plt.ylabel("Residuals")
plt.legend(loc = "upper left")
plt.plot([0, 1.1], [0, 0], c = "black")

plt.show()

In [ ]:
# Plot predictions (real values)

plt.scatter(train_pre, Ytrain, c = "red",  label = "Training data")
plt.scatter(test_pre, Ytest, c = "green",  label = "Validation data")
plt.title("Linear regression")
plt.xlabel("Predicted values")
plt.ylabel("Real values")
plt.legend(loc = "upper left")
plt.plot([0.1, 0.95], [0, 1], c = "black")
plt.show()

## 7. Ridge Regression
 - Now, we will use a regression called Ridge. It is the most commonly used regression algorithm to approximate an answer for an equation with no unique solution.

In [ ]:
# Defining list of alphas

alphas = [0.00025, 0.0003, 0.0005, 0.0007, 0.001, 0.002, 0.0025, 0.003, 0.004, 0.005, 0.006,
          0.007, 0.008, 0.009, 0.01, 0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 1.1, 1.5, 2]

In [ ]:
# Setting alphas and fitting with ridge

ridge = RidgeCV(alphas = alphas)
ridge.fit(Xtrain,Ytrain)

In [ ]:
alpha = ridge.alpha_
print('The best alpha found was: ',alpha)

In [ ]:
print("Try again for more precision with alphas centered around " + str(alpha))
ridge = RidgeCV(alphas = [alpha * .6, alpha * .65, alpha * .7, alpha * .75, alpha * .8, alpha * .85, 
                          alpha * .9, alpha * .95, alpha, alpha * 1.05, alpha * 1.1, alpha * 1.15,
                          alpha * 1.25, alpha * 1.3, alpha * 1.35, alpha * 1.4],cv = 5)
ridge.fit(Xtrain, Ytrain)
alpha = ridge.alpha_
print("Best alpha : ", alpha)

In [ ]:
model_ridge = Ridge()
cv_ridge = [rmse_CV_train(Ridge(alpha = alpha)).mean() 
            for alpha in alphas]
cv_ridge = pd.Series(cv_ridge, index = alphas)
cv_ridge.plot(title = "Validation")
plt.xlabel("alpha")
plt.ylabel("rmse")
plt.xlim((0.00225, 0.0026))
plt.ylim((0.0853, 0.0855))

- Note the U-ish shaped curve above. When alpha is too large the regularization is too strong and the model cannot capture all the complexities in the data. If however we let the model be too flexible (alpha small) the model begins to overfit.

In [ ]:
# Calculating the Ridge RMSE

print("Ridge RMSE on Training set :", rmse_CV_train(ridge).mean())
print("Ridge RMSE on Test set :", rmse_CV_test(ridge).mean())

In [ ]:
score = cross_val_score(ridge,Xtrain, Ytrain, cv = 5)
print('Score is: '+ str(np.mean(score)))

In [ ]:
Ytrain_ridge = ridge.predict(Xtrain)
Ytest_ridge = ridge.predict(Xtest)

In [ ]:
# Plot between predicted values and residuals

plt.scatter(Ytrain_ridge, Ytrain_ridge -Ytrain, c = "red",  label = "Training data")
plt.scatter(Ytest_ridge, Ytest_ridge -Ytest, c = "green", marker = "v", label = "Validation data")
plt.title("Linear regression with Ridge regularization")
plt.xlabel("Predicted values")
plt.ylabel("Residuals")
plt.legend(loc = "upper left")
plt.plot([0, 1.2], [-0.6, 0.6], c = "black")
plt.show()

In [ ]:
# Plot predictions - Real values

plt.scatter(Ytrain_ridge, Ytrain, c = "red",  label = "Training data")
plt.scatter(Ytest_ridge, Ytest, c = "green",  label = "Validation data")
plt.title("Linear regression with Ridge regularization")
plt.xlabel("Predicted values")
plt.ylabel("Real values")
plt.legend(loc = "upper left") 
plt.plot([0.27, 0.93], [0.2, 1], c = "black")
plt.show()

## 8. LASSO Regression
 - In this section we will use an especific algorithm called LASSO. It is a regression analysis method that performs both variable selection and regularization in order to enhance the prediction accuracy and interpretability of the statistical model it produces.

In [ ]:
# Defining another list of alphas

alphas2 = [0.025, 0.03, 0.05, 0.07, 0.1, 0.15, 0.2, 0.25, 0.3, 1, 1.25, 1.5, 1.75, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 5,
           10, 13, 15, 17, 20, 23, 25, 30, 35, 40, 50]

In [ ]:
# Setting alphas and fitting with lasso

lasso = LassoCV(alphas = alphas2).fit(Xtrain, Ytrain)

In [ ]:
alpha2 = lasso.alpha_
print('The best alpha found was: ',alpha2)

In [ ]:
print("Try again for more precision with alphas centered around " + str(alpha2))
lasso = LassoCV(alphas = [alpha2 * .6, alpha2 * .65, alpha2 * .7, alpha2 * .75, alpha2 * .8, alpha2 * .85, 
                          alpha2 * .9, alpha2 * .95, alpha2, alpha2 * 1.05, alpha2 * 1.1, alpha2 * 1.15,
                          alpha2 * 1.25, alpha2 * 1.3, alpha2 * 1.35, alpha2 * 1.4],cv = 5)
lasso.fit(Xtrain, Ytrain)
alpha2 = lasso.alpha_
print("Best alpha : ", alpha2)

In [ ]:
# Calculating the Ridge RMSE

print("Lasso RMSE on Training set :", rmse_CV_train(lasso).mean())
print("Lasso RMSE on Test set :", rmse_CV_test(lasso).mean())

## 9. Elastic Net
- In the fitting of linear or logistic regression models, the elastic net is a regularized regression method that linearly combines the lasso and ridge methods. 

In [ ]:
#Setting Alphas and L1-ratio

alphas = [0.0005, 0.001, 0.01, 0.03, 0.05, 0.1]
l1_ratios = [1.5, 1.1, 1, 0.9, 0.8, 0.7, 0.5]
ENet = ElasticNet()

In [ ]:
# Defining the CV Eastic Net

cv_elastic = [rmse_CV_train(ElasticNet(alpha = alpha, l1_ratio=l1_ratio)).mean() 
            for (alpha, l1_ratio) in product(alphas, l1_ratios)]

In [ ]:
# Plot the relation between the alphas, l1 and the RMSE

matplotlib.rcParams['figure.figsize'] = (12.0, 6.0)
idx = list(product(alphas, l1_ratios))
p_cv_elastic = pd.Series(cv_elastic, index = idx)
p_cv_elastic.plot(title = "Validation")
plt.xlabel("Alpha - L1_ratio")
plt.ylabel("RMSE")

In [ ]:
# Let's zoom in to the first 10 parameter pairs

matplotlib.rcParams['figure.figsize'] = (12.0, 6.0)
idx = list(product(alphas, l1_ratios))[:10]
p_cv_elastic = pd.Series(cv_elastic[:10], index = idx)
p_cv_elastic.plot(title = "Validation")
plt.xlabel("Alpha - L1_ratio")
plt.ylabel("RMSE")

In [ ]:
score = rmse_CV_train(ENet)
print("Elastic Net RMSE on Train set: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

In [ ]:
elastic = ElasticNet(alpha=0.0005, l1_ratio=0.9)
elastic.fit(Xtrain, Ytrain)
ElasticNet(alpha=0.0005, copy_X=True, fit_intercept=True, l1_ratio=0.9,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

In [ ]:
# Let's look at the residuals
matplotlib.rcParams['figure.figsize'] = (6.0, 6.0)

preds = pd.DataFrame({"preds":elastic.predict(Xtrain), "true":Ytrain})
preds["residuals"] = preds["true"] - preds["preds"]
preds.plot(x = "preds", y = "residuals",kind = "scatter")

In [ ]:
score = cross_val_score(elastic,Xtrain, Ytrain, cv = 5)
print('Score is: '+ str(np.mean(score)))

In [ ]:
print('R^2 train: %.3f' %  r2_score(preds['true'], preds['preds']))

In [ ]:
coef = pd.Series(elastic.coef_, index = Xtrain.columns)
print("Elastic Net picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

In [ ]:
imp_coef = pd.concat([coef.sort_values().head(10),
                     coef.sort_values().tail(10)])

In [ ]:
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Coefficients in the Elastic Net Model")

## 10. Random Forest Regressor
 - Random forests or random decision forests are an ensemble learning method for classification, regression and other tasks, that operate by constructing a multitude of decision trees at training time and outputting the class that is the mode of the classes (classification) or mean prediction (regression) of the individual trees.

In [ ]:
RF = RandomForestRegressor(max_depth=30, n_estimators=500, max_features = 15, oob_score=True, random_state=1234)
score = cross_val_score(RF,Xtrain, Ytrain, cv = 5, n_jobs = -1)
print("Random Forest RMSE on Training set :", rmse_CV_train(RF).mean())
print("Random Forest RMSE on Test set :", rmse_CV_test(RF).mean())
print('Score is: '+ str(np.mean(score)))

## 11. KNN (k-nearest neighbors)
 - For now, let's try the k-nearest neighbors algorithm (k-NN) in regression. This output is the property value for the object. This value is the average of the values of its k nearest neighbors.

 - Then, let's try some numbers to the K and CV and find the best performance

In [ ]:
# Setting the K numbers

neighbors = [3,5,7,9,11,13,15,20,25,30]

In [ ]:
# Here, we define a function that test (with CV equal to 3) various K-nn and yours misclassification error

print('With CV = 3')
cv_scores = []

for k in neighbors:
    knn = KNeighborsRegressor(n_neighbors=k,weights='uniform')
    scores = cross_val_score(knn, Xtrain, Ytrain, cv=3)
    cv_scores.append(scores.mean())
    
    
MSE = [1 - x for x in cv_scores]
optimal_k = neighbors[MSE.index(min(MSE))]
print("The optimal number of neighbors is %d" % optimal_k)

In [ ]:
# Plot a graph that show us the perfomance of the K-nn with each number of neighbors (in this case, the CV is equal to 3)

plt.plot(neighbors, MSE)
plt.xlabel('Number of Neighbors K')
plt.ylabel('Misclassification Error')
plt.show()

In [ ]:
# Here, we define a function that test (with CV equal to 5) various K-nn and yours misclassification error

print('With CV = 5')
cv_scores = []

for k in neighbors:
    knn = KNeighborsRegressor(n_neighbors=k,weights='uniform')
    scores = cross_val_score(knn, Xtrain, Ytrain, cv=5)
    cv_scores.append(scores.mean())
    
    
MSE = [1 - x for x in cv_scores]
optimal_k = neighbors[MSE.index(min(MSE))]
print("The optimal number of neighbors is %d" % optimal_k)

In [ ]:
# Plot a graph that show us the perfomance of the K-nn with each number of neighbors (in this case, the CV is equal to 5)

plt.plot(neighbors, MSE)
plt.xlabel('Number of Neighbors K')
plt.ylabel('Misclassification Error')
plt.show()

In [ ]:
# Here, we define a function that test (with CV equal to 7) various K-nn and yours misclassification error

print('With CV = 7')
cv_scores = []

for k in neighbors:
    knn = KNeighborsRegressor(n_neighbors=k,weights='uniform')
    scores = cross_val_score(knn, Xtrain, Ytrain, cv=7)
    cv_scores.append(scores.mean())
    
    
MSE = [1 - x for x in cv_scores]
optimal_k = neighbors[MSE.index(min(MSE))]
print("The optimal number of neighbors is %d" % optimal_k)

In [ ]:
# Plot a graph that show us the perfomance of the K-nn with each number of neighbors (in this case, the CV is equal to 7)

plt.plot(neighbors, MSE)
plt.xlabel('Number of Neighbors K')
plt.ylabel('Misclassification Error')
plt.show()

In [ ]:
# Here, we define a function that test (with CV equal to 9) various K-nn and yours misclassification error

print('With CV = 9')
cv_scores = []

for k in neighbors:
    knn = KNeighborsRegressor(n_neighbors=k,weights='uniform')
    scores = cross_val_score(knn, Xtrain, Ytrain, cv=9)
    cv_scores.append(scores.mean())
    
    
MSE = [1 - x for x in cv_scores]
optimal_k = neighbors[MSE.index(min(MSE))]
print("The optimal number of neighbors is %d" % optimal_k)

In [ ]:
# Plot a graph that show us the perfomance of the K-nn with each number of neighbors (in this case, the CV is equal to 9)

plt.plot(neighbors, MSE)
plt.xlabel('Number of Neighbors K')
plt.ylabel('Misclassification Error')
plt.show()

In [ ]:
# Here, we define a function that test (with CV equal to 11) various K-nn and yours misclassification error

print('With CV = 11')
cv_scores = []

for k in neighbors:
    knn = KNeighborsRegressor(n_neighbors=k,weights='uniform')
    scores = cross_val_score(knn, Xtrain, Ytrain, cv=11)
    cv_scores.append(scores.mean())
    
    
MSE = [1 - x for x in cv_scores]
optimal_k = neighbors[MSE.index(min(MSE))]
print("The optimal number of neighbors is %d" % optimal_k)

In [ ]:
# Plot a graph that show us the perfomance of the K-nn with each number of neighbors (in this case, the CV is equal to 11)

plt.plot(neighbors, MSE)
plt.xlabel('Number of Neighbors K')
plt.ylabel('Misclassification Error')
plt.show()

In [ ]:
# Here, we define a function that test (with CV equal to 13) various K-nn and yours misclassification error

print('With CV = 13')
cv_scores = []

for k in neighbors:
    knn = KNeighborsRegressor(n_neighbors=k,weights='uniform')
    scores = cross_val_score(knn, Xtrain, Ytrain, cv=13)
    cv_scores.append(scores.mean())
    
    
MSE = [1 - x for x in cv_scores]
optimal_k = neighbors[MSE.index(min(MSE))]
print("The optimal number of neighbors is %d" % optimal_k)

In [ ]:
# Plot a graph that show us the perfomance of the K-nn with each number of neighbors (in this case, the CV is equal to 13)

plt.plot(neighbors, MSE)
plt.xlabel('Number of Neighbors K')
plt.ylabel('Misclassification Error')
plt.show()

In [ ]:
# Here, we define a function that test (with CV equal to 15) various K-nn and yours misclassification error

print('With CV = 15')
cv_scores = []

for k in neighbors:
    knn = KNeighborsRegressor(n_neighbors=k,weights='uniform')
    scores = cross_val_score(knn, Xtrain, Ytrain, cv=15)
    cv_scores.append(scores.mean())
    
    
MSE = [1 - x for x in cv_scores]
optimal_k = neighbors[MSE.index(min(MSE))]
print("The optimal number of neighbors is %d" % optimal_k)

In [ ]:
# Plot a graph that show us the perfomance of the K-nn with each number of neighbors (in this case, the CV is equal to 15)

plt.plot(neighbors, MSE)
plt.xlabel('Number of Neighbors K')
plt.ylabel('Misclassification Error')
plt.show()

In [ ]:
# Here, we define a function that test (with CV equal to 20) various K-nn and yours misclassification error

print('With CV = 20')
cv_scores = []

for k in neighbors:
    knn = KNeighborsRegressor(n_neighbors=k,weights='uniform')
    scores = cross_val_score(knn, Xtrain, Ytrain, cv=20)
    cv_scores.append(scores.mean())
    
    
MSE = [1 - x for x in cv_scores]
optimal_k = neighbors[MSE.index(min(MSE))]
print("The optimal number of neighbors is %d" % optimal_k)

In [ ]:
# Plot a graph that show us the perfomance of the K-nn with each number of neighbors (in this case, the CV is equal to 20)

plt.plot(neighbors, MSE)
plt.xlabel('Number of Neighbors K')
plt.ylabel('Misclassification Error')
plt.show()

In [ ]:
# Here, we define a function that test (with CV equal to 25) various K-nn and yours misclassification error

print('With CV = 25')
cv_scores = []

for k in neighbors:
    knn = KNeighborsRegressor(n_neighbors=k,weights='uniform')
    scores = cross_val_score(knn, Xtrain, Ytrain, cv=25)
    cv_scores.append(scores.mean())
    
    
MSE = [1 - x for x in cv_scores]
optimal_k = neighbors[MSE.index(min(MSE))]
print("The optimal number of neighbors is %d" % optimal_k)

In [ ]:
# Plot a graph that show us the perfomance of the K-nn with each number of neighbors (in this case, the CV is equal to 25)

plt.plot(neighbors, MSE)
plt.xlabel('Number of Neighbors K')
plt.ylabel('Misclassification Error')
plt.show()

In [ ]:
# Here, we define a function that test (with CV equal to 30) various K-nn and yours misclassification error

print('With CV = 30')
cv_scores = []

for k in neighbors:
    knn = KNeighborsRegressor(n_neighbors=k,weights='uniform')
    scores = cross_val_score(knn, Xtrain, Ytrain, cv=30)
    cv_scores.append(scores.mean())
    
    
MSE = [1 - x for x in cv_scores]
optimal_k = neighbors[MSE.index(min(MSE))]
print("The optimal number of neighbors is %d" % optimal_k)

In [ ]:
# Plot a graph that show us the perfomance of the K-nn with each number of neighbors (in this case, the CV is equal to 30)

plt.plot(neighbors, MSE)
plt.xlabel('Number of Neighbors K')
plt.ylabel('Misclassification Error')
plt.show()

 - We can conclude that our best perfomace with K-NN was with CV equal to 20 and the optimal number of neighbors is 9. So, let's use it and take more useful info of your classification.

In [ ]:
# Set the K-nn with K=7 and CV=20

knn = KNeighborsRegressor(n_neighbors=9)
knn.fit(Xtrain,Ytrain)
scores = cross_val_score(knn, Xtrain, Ytrain, cv=20).mean()
print(scores)

## 12. Conclusion

 - After we analyze all the data and look at various methods of classification, we concluded that we will predict our "testTOsubmit" with the Random Forest Regressor.

 - Now, let's predict in our test file and send the results

In [ ]:
# Open the file that contains a sample how to send ours predictions

sample = pd.read_csv('../input/atividade-3-pmr3508/sample_sub_1.csv')
print ("submission size: ", sample.shape)

In [ ]:
sample.head()

 - Here we will adjust the 'Id' and 'median_house_value' to the predictions

In [ ]:
Ytrain['median_house_value'] = trainMEDIAN
Ytest['median_house_value'] = testMEDIAN

In [ ]:
Y = Ytrain + Ytest

In [ ]:
Xtrain['Id'] = trainID
Xtest['Id'] = testID

In [ ]:
X = Xtrain + Xtest

In [ ]:
testTOsubmit['Id'] = testTOsubmitID

In [ ]:
# Let's fit with thw Random Forest

RF.fit(X,Y)

In [ ]:
# Drop the target column with NaN values

testTOsubmit = testTOsubmit.dropna(axis=1, how='all')

In [ ]:
predictionsTOsubmit = RF.predict(testTOsubmit)

In [ ]:
# Sending the predictions

str(predictionsTOsubmit)
ids = testTOsubmitID
submission = pd.DataFrame({'Id':ids,'median_house_value':predictionsTOsubmit[:]})
submission.to_csv("predictionsT3.csv", index = False)

In [ ]:
submission.shape

In [ ]:
submission

## References


 - [Cross Validation Score](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html)
 - [Confusion Matrix](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)
 - [K-Nearest Neighbors](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html)
 - [Naive Bayes](http://scikit-learn.org/stable/modules/naive_bayes.html)
 - [Pandas](https://pandas.pydata.org/pandas-docs/stable/)
 - [ROC Curve](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html)
 - [Linear Regression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)
 - [Geopy](https://github.com/geopy/geopy)
 - [Geospatial Feature Engineering and Visualization](https://www.kaggle.com/camnugent/geospatial-feature-engineering-and-visualization)
 - [Lasso Regression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)
 - [Split Data](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html)
 - [Ridge Regression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html)
 - [Wikipedia](https://pt.wikipedia.org/wiki/Wikipédia:Página_principal)
 - [Random Forest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)